In [4]:
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
import copy
import math
import time
from pathlib import Path
from collections import Counter

# Loading trace : Needs to expanded into 4K chunks
path = r'/Users/chandranilchakraborttii/Documents/GC_pred/data'
all_files = glob.glob(os.path.join(path, "synthetic_dataprep_A_with_deathtime__5.0GB_SSD.csv"))

f = all_files[0]  # Change the file name as required
print("File Name :" + str(f))

df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
cols = ['LBA','deathtime']
df.columns = cols
print("Before",len(df))
df['deathtime'] = df['deathtime'].replace(np.NaN, -1)
df = df.loc[df['deathtime'] != -1]
print("After",len(df))

lba_list = df['LBA'].tolist()
deathtime_list = df['deathtime'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

File Name :/Users/chandranilchakraborttii/Documents/GC_pred/data/synthetic_dataprep_A_with_deathtime__5.0GB_SSD.csv
Before 13107210
After 13107210
Min LBA in the dataset : 1
Max LBA in the dataset : 1310721
Number of unique LBAs in the data : 1310721
Number of IO Accesses : 13107210


In [5]:
#SSD specifications
num_page_addresses = len(Counter(df['LBA']))
page_size = 4096
page_per_block = 16
GB = 1024*1024*1024
SSD_size = num_page_addresses*page_size
# SSD_size_GB_normal = round_decimals_up(SSD_size/GB,1)
SSD_size_GB_normal = SSD_size/GB
over_provisioning_ratio = 0.2
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))
# SSD_size_full = round_decimals_up((1 + over_provisioning_ratio)*SSD_size_GB_normal,1)
SSD_size_full = (1 + over_provisioning_ratio)*SSD_size_GB_normal
print("SSD Capacity (Available in GB) :",SSD_size_GB_normal)
print("SSD Capacity (Total in GB)     :",SSD_size_full)

SSD Capacity (Available in GB) : 5.000003814697266
SSD Capacity (Total in GB)     : 6.000004577636719


In [6]:
GB = 1024*1024*1024
ssd_capacity = SSD_size_full *GB

# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)

block_struct = {}
for x in free_list_block:
    start_lba = x
    write_ptr=0
    invalid_pages=0
    death_time = 0
    valid_bitmap = []
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)

    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    block_struct[start_lba]=segment
    

print("Total number of Blocks created: ", len(block_addresses))
print("Total number of Pages created:  ", len(page_addresses))
print(str(len(block_struct)) + " 4K blocks Initialized" )

Total number of Blocks created:  98049
Total number of Pages created:   1568771
98049 4K blocks Initialized


In [7]:
def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][2][prev_page] = False                          # Setting bitmap to False
    block_struct[prev_block][1] = block_struct[prev_block][2].count(True)     # Setting invalid pages
    L2P.pop(lba)


    
#map LBA to phys
def map_lba(lba,deathtime, block_IO_burst):
    # Finding which block to add the LBA
    block_map={}
    found = False
    for x in block_IO_burst:
        # If the death Time has passed, make it priority 
        if(block_struct[x][5] == True):
            block_select = x
            found = True 
        else:
            block_map[x]= block_struct[x][4]
    # Finding the block with closest death time 
    if(found !=True):
        delta = max(deathtime_range_list)*100
        keys = list(block_map.keys())
        block_select = -1
        for x in keys:
            tmp = abs(block_map[x] - deathtime)
            if(tmp < delta):
                delta = tmp
                block_select = x
    
    if(block_struct[block_select][5] == True):
        priority_writes[0] = priority_writes[0] + 1
        
    # Block Found, now updating block          
    phys_addr = block_struct[block_select][0] + (block_struct[block_select][3])
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[block_select][2][block_struct[block_select][3]] = True             # Setting Bitmap
    block_struct[block_select][1] = block_struct[block_select][2].count(True)            # Setting invalid pages
    block_struct[block_select][3] = block_struct[block_select][3] + 1               # Increasing Write pointer
    total_writes[0] = total_writes[0] + 1

    

#check if we need to close/open block. Do not perform GC if we are already
def check_GC (block_IO_burst, in_gc):
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] == page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    if(len(free_list_block) == 0):
        print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
   # Checking if GC is needed
    elif (len(free_list_block) <= GC_THRESHOLD):
        # Checking if GC is already going on
        if(in_gc != True):
            in_gc = do_greedy_gc(block_IO_burst,in_gc) 
    return block_IO_burst




def do_greedy_gc(block_IO_burst,in_gc):
#     print(counter)
    in_gc = True
    gc_writes = 0 
    min_val = float('inf')
    for x in closed_blocks:              
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
            gc_blk = x
        # For each closed block, check phys_addr: If valid bitmap is True (data is valid), copy to OP capacity
#     print(min_val)
#     print(block_struct[gc_blk])
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        # Updating P2L
        if (phys_addr in P2L):
            gc_lba = P2L[phys_addr]
            P2L.pop(phys_addr)            
        # Updating L2P
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        death_time = block_struct[prev_block][4]                      # Getting current death_time for the block
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(gc_lba)
            total_gc_writes[0] = total_gc_writes[0] + 1
            #check if we need to get a new block
            block_IO_burst = check_GC(block_IO_burst,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(gc_lba,death_time,block_IO_burst)
            block_IO_burst = decrease_death_time(block_IO_burst)  
    
                
    if(gc_writes > 64):
        print("GC writes not as expected", gc_writes)
    total_gc_writes[0] = total_gc_writes[0] + gc_writes
    
    invalid_pages = 0
    valid_bitmap = []
    write_ptr = 0
    death_time = 0
    priority = False
    death_time_original = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)
            
    # Reseting the GC blk and add to free_block_list
    block_struct[gc_blk]= [gc_blk,invalid_pages,valid_bitmap,write_ptr,death_time,priority,death_time_original]
    closed_blocks.remove(gc_blk)
    free_list_block.append(gc_blk)
    for x in block_IO_burst:
        # If block is full, close block and reset death time
        if(block_struct[x][3] >= page_per_block):
            death_time = block_struct[x][6]               # Copying original death time to be set in the new block
            closed_blocks.append(x)                       # Adding to closed list
            block_IO_burst.remove(x)                    
            new_block = free_list_block.pop(0)            #  Requesting a new block
            block_IO_burst.append(new_block)              # Adding to open blocks
            block_struct[new_block][4] = death_time       # Setting death time counter of the new block
            block_struct[new_block][6] = death_time       # Setting death time of the new block
    
    in_gc = False
    return in_gc

def decrease_death_time(block_IO_burst):
    # Decreasing Death Time for each block
    s_flag = True
    for x in block_IO_burst:
        # If death time passed, make the block priority 
        # Priority means: All subsequent IOs will be added to this block here until full
        if (block_struct[x][4] <= 0):
            block_struct[x][5] = True
        else:
            block_struct[x][4] = block_struct[x][4] - 1  
    return block_IO_burst

In [8]:
# Setting global parameters
# Initalizing Starting Free Blocks..
global gc_writes
global in_gc 

num_cur_blocks_open = 20        # Hyperparameter

L2P = {}
P2L = {}
closed_blocks = []
cur_blocks_open = []
lba_burst = []
deathtime_range_list = []
interval = float(100/num_cur_blocks_open)
gc_writes = 0
in_gc = False
death_time_passed = []

count = 0
while(count < num_cur_blocks_open):
    if(count <= num_cur_blocks_open/4):
        deathtime_range_list.append(int(np.percentile(deathtime_list, (count+1)*interval*2)))
        #print(int(np.percentile(deathtime_list, (count+1)*interval*2)))
    else:
        deathtime_range_list.append(int(np.percentile(deathtime_list, 49.9 + (count)*interval/2)))
        #print(int(np.percentile(deathtime_list, 50 + (count+1)*interval/2)))
    count = count + 1
    

block_IO_burst = []
death_time_ranges = []
print("Initalizing Starting Free Blocks...")
for x in range(num_cur_blocks_open):
    block_num = free_list_block.pop(0)                                            # Getting a free block
    block_IO_burst.append(block_num) 
    block_struct[block_num][4] = deathtime_range_list[x]                          # Setting death time
    block_struct[block_num][6] = deathtime_range_list[x]  

Initalizing Starting Free Blocks...


In [9]:
total_writes = []
total_writes.append(0)
total_gc_writes = []
total_gc_writes.append(0)
priority_writes = []
priority_writes.append(0)
total_user_writes = []
total_user_writes.append(0)
blocks_closed_early = []
counter = 0
GC_THRESHOLD = 50

print("Starting Trace..!")
start_time = time.time()
while(counter < len(lba_list)):
    if(counter >100000 and counter%100000==0):
        print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
    lba= int(lba_list[counter])
    death_time = int(deathtime_list[counter])
    if lba in L2P:
        invalidate_lba(lba)
    block_IO_burst = check_GC(block_IO_burst,in_gc)
    map_lba(lba,death_time,block_IO_burst)
    block_IO_burst = decrease_death_time(block_IO_burst)
    counter = counter + 1

end_time = time.time()
run_time = end_time - start_time
print("GC Threshold :",GC_THRESHOLD)
print("OverProvisioning Ratio :",over_provisioning_ratio)
print("Number of Open Blocks :",num_cur_blocks_open)
print("Total Number of writes :",total_writes[0])
print("Total Number of GC writes :",total_gc_writes[0])
print("Total Number of Priority writes    :",priority_writes[0])
print("Total Number of user writes :",(total_writes[0] - total_gc_writes[0]))
print("Write amplification",total_gc_writes[0]/(total_writes[0] - total_gc_writes[0]))
print("Priority writes ratio to total writes",priority_writes[0]/total_writes[0])
print("Execution Time for the FTL :",run_time)

Starting Trace..!
Percentage completed in (%)  : 1.52587774209767
Percentage completed in (%)  : 2.288816613146505
Percentage completed in (%)  : 3.05175548419534
Percentage completed in (%)  : 3.8146943552441748
Percentage completed in (%)  : 4.57763322629301
Percentage completed in (%)  : 5.340572097341845
Percentage completed in (%)  : 6.10351096839068
Percentage completed in (%)  : 6.866449839439515
Percentage completed in (%)  : 7.6293887104883495
Percentage completed in (%)  : 8.392327581537184
Percentage completed in (%)  : 9.15526645258602
Percentage completed in (%)  : 9.918205323634854
Percentage completed in (%)  : 10.68114419468369
Percentage completed in (%)  : 11.444083065732524
Percentage completed in (%)  : 12.20702193678136
Percentage completed in (%)  : 12.969960807830194
Percentage completed in (%)  : 13.73289967887903
Percentage completed in (%)  : 14.495838549927864
Percentage completed in (%)  : 15.258777420976699
Percentage completed in (%)  : 16.021716292025534


In [10]:
print("Done..!")

Done..!
